In [173]:
# Para llevar el registro
!pip install -q comet_ml
import comet_ml
comet_ml.init(project_name = 'FineTunning')

In [174]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, Conv2D, MaxPooling2D, Concatenate, PReLU
from tensorflow.keras.optimizers import RMSprop, SGD, Adam, Adadelta
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.regularizers import l2

In [175]:
# Se crea el experimento en Comet
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging = True,
    auto_histogram_gradient_logging = True,
    auto_histogram_activation_logging = True,
    log_code = True
)
experiment.set_name('Test 19')

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/finetunning/7e736e55b18b46248fffda42e93c9d05
COMET INFO:   Others:
COMET INFO:     Name : Test 19
COMET INFO:   Parameters:
COMET INFO:     Optimizer                       : RMSprop
COMET INFO:     RMSprop_centered                : False
COMET INFO:     RMSprop_clipnorm                : None
COMET INFO:     RMSprop_clipvalue               : None
COMET INFO:     RMSprop_ema_momentum            : 0.99
COMET INFO:     RMSprop_ema_overwrite_frequency : 100
COMET INFO:     RMSprop_epsilon                 : 1e-07
COMET INFO:     RMSprop_global_clipnorm         : None
COMET INFO:     RMSprop_is_legacy_optimizer     : Fals

In [176]:
# Parámetros
i_width = int(178 * 1.2)
i_height = int(218 * 1.2)
input_shape = (i_width, i_height, 3)

parameters = {
    'num_class' : 2,
    'epochs' : 200,
    'batch_size' : 16,
    'learning_rate' : 0.001,
    'loss' : 'binary_crossentropy',
    'optimizer' : 'RMSprop'}

experiment.log_parameters(parameters)

In [177]:
# Loading images
train_dir = '/kaggle/input/data-finetunning/Norma_align/Norma_align/train'
val_dir = '/kaggle/input/data-finetunning/Norma_align/Norma_align/val'

# Image Generator
gen_train = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range = 0.15,  # % de desplazamiento horizontal
    height_shift_range = 0.15, # % de desplazamiento vertical
    brightness_range = [0.25, 1.25], # Porcentaje de brillo en el rango
    zoom_range = 0.3, # Zoom de hasta 30%
    channel_shift_range = 20, # Cada pixel de cada canal varía su intensidad en una vecindad de 20 unidades
    fill_mode = 'nearest', # Después de desplazamientos que dejan la imágen 'incompleta' se rellena con el valor de los pixeles más cercanos
    horizontal_flip = True,
    vertical_flip = True,
    rescale = 1./255.)
train = gen_train.flow_from_directory(
    train_dir,
    target_size = (i_width, i_height),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = parameters['batch_size'],
    shuffle = True)

gen_val = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range = 0.15,
    height_shift_range = 0.15,
    brightness_range = [0.25, 1.25],
    zoom_range = 0.3,
    channel_shift_range = 20,
    fill_mode = 'nearest',
    horizontal_flip = True,
    vertical_flip = True,
    rescale = 1./255.)
val = gen_val.flow_from_directory(
    val_dir,
    target_size = (i_width, i_height),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = parameters['batch_size'],
    shuffle = True)

Found 120 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [178]:
# Loading model
path_load = os.path.join('/kaggle/input/data-finetunning/best_val_loss.h5')
model_loaded = tf.keras.models.load_model(path_load)

model_0 = Sequential()

for layer in model_loaded.layers[:15]:
    layer.trainable = False
    model_0.add(layer)

# model_0.add(Dense(128, activation = 'relu', kernel_regularizer = l2(0.01)))
model_0.add(Dense(512, activation = 'PReLU'))
model_0.add(Dense(256, activation = 'PReLU'))
# model_0.add(Dropout(0.12))
model_0.add(Dense(128, activation = 'relu', kernel_regularizer = l2(0.0055)))
model_0.add(Dropout(0.5))
model_0.add(Dense(64, activation = 'relu', kernel_regularizer = l2(0.0055)))
model_0.add(Dense(1, activation = 'sigmoid'))
    
# model_0.summary()

In [179]:
# Checkpoint para el mejor modelo (val_accuracy)
filepath = 'best_val_loss.h5'
checkpoint = ModelCheckpoint(
    filepath,
    monitor = 'val_loss',
    verbose = 1,
    save_best_only = True,
    mode = 'min')

In [180]:
# Compilación y entrenamiento
model_0.compile(
    loss = parameters['loss'],
    optimizer = RMSprop(learning_rate = parameters['learning_rate']),
    metrics = ['accuracy'])

model_0.fit(
    train,
    epochs = parameters['epochs'],
    verbose = 1,
    validation_data = val,
    callbacks = [checkpoint])

Epoch 1/200
8/8 [==============================] - ETA: 0s - loss: 1.9957 - accuracy: 0.5500
Epoch 1: val_loss improved from inf to 1.88471, saving model to best_val_loss.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 8s 674ms/step - loss: 1.9957 - accuracy: 0.5500 - val_loss: 1.8847 - val_accuracy: 0.5750
Epoch 2/200
8/8 [==============================] - ETA: 0s - loss: 1.7420 - accuracy: 0.6750
Epoch 2: val_loss improved from 1.88471 to 1.68151, saving model to best_val_loss.h5
8/8 [==============================] - 6s 715ms/step - loss: 1.7420 - accuracy: 0.6750 - val_loss: 1.6815 - val_accuracy: 0.7000
Epoch 3/200
8/8 [==============================] - ETA: 0s - loss: 1.5762 - accuracy: 0.6667
Epoch 3: val_loss did not improve from 1.68151
8/8 [==============================] - 4s 535ms/step - loss: 1.5762 - accuracy: 0.6667 - val_loss: 1.6870 - val_accuracy: 0.7500
Epoch 4/200
8/8 [==============================] - ETA: 0s - loss: 1.4986 - accuracy: 0.7250
Epoch 4: val_loss did not improve from 1.68151
8/8 [==============================] - 5s 694ms/step - loss: 1.4986 - accuracy: 0.7250 - val_loss: 1.7584 - val_accuracy: 0.5500
Epoch 5/200
8/8 [=========

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 4s 492ms/step - loss: 1.3579 - accuracy: 0.7000 - val_loss: 1.4370 - val_accuracy: 0.7000
Epoch 6/200
8/8 [==============================] - ETA: 0s - loss: 1.2290 - accuracy: 0.7417
Epoch 6: val_loss improved from 1.43696 to 1.29260, saving model to best_val_loss.h5
8/8 [==============================] - 4s 505ms/step - loss: 1.2290 - accuracy: 0.7417 - val_loss: 1.2926 - val_accuracy: 0.6750
Epoch 7/200
8/8 [==============================] - ETA: 0s - loss: 1.1869 - accuracy: 0.7583
Epoch 7: val_loss did not improve from 1.29260
8/8 [==============================] - 3s 438ms/step - loss: 1.1869 - accuracy: 0.7583 - val_loss: 1.3603 - val_accuracy: 0.6000
Epoch 8/200
8/8 [==============================] - ETA: 0s - loss: 1.1211 - accuracy: 0.7500
Epoch 8: val_loss improved from 1.29260 to 1.15424, saving model to best_val_loss.h5
8/8 [==============================] - 4s 451ms/step - loss: 1.1211 - accuracy: 0.7500 - val_loss: 1.1542 - val_accur

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 4s 444ms/step - loss: 0.8476 - accuracy: 0.8167 - val_loss: 0.8964 - val_accuracy: 0.7750
Epoch 13/200
8/8 [==============================] - ETA: 0s - loss: 0.8356 - accuracy: 0.7583
Epoch 13: val_loss did not improve from 0.89644
8/8 [==============================] - 3s 434ms/step - loss: 0.8356 - accuracy: 0.7583 - val_loss: 1.0140 - val_accuracy: 0.6000
Epoch 14/200
8/8 [==============================] - ETA: 0s - loss: 0.8000 - accuracy: 0.8083
Epoch 14: val_loss did not improve from 0.89644
8/8 [==============================] - 3s 436ms/step - loss: 0.8000 - accuracy: 0.8083 - val_loss: 0.9619 - val_accuracy: 0.7250
Epoch 15/200
8/8 [==============================] - ETA: 0s - loss: 0.8165 - accuracy: 0.7583
Epoch 15: val_loss did not improve from 0.89644
8/8 [==============================] - 3s 442ms/step - loss: 0.8165 - accuracy: 0.7583 - val_loss: 1.0701 - val_accuracy: 0.6250
Epoch 16/200
8/8 [==============================] - ETA: 0

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 3s 426ms/step - loss: 0.8174 - accuracy: 0.7667 - val_loss: 0.8853 - val_accuracy: 0.7250
Epoch 17/200
8/8 [==============================] - ETA: 0s - loss: 0.8682 - accuracy: 0.7250
Epoch 17: val_loss did not improve from 0.88526
8/8 [==============================] - 3s 421ms/step - loss: 0.8682 - accuracy: 0.7250 - val_loss: 0.8975 - val_accuracy: 0.7000
Epoch 18/200
8/8 [==============================] - ETA: 0s - loss: 0.6930 - accuracy: 0.8250
Epoch 18: val_loss did not improve from 0.88526
8/8 [==============================] - 3s 425ms/step - loss: 0.6930 - accuracy: 0.8250 - val_loss: 1.1020 - val_accuracy: 0.6750
Epoch 19/200
8/8 [==============================] - ETA: 0s - loss: 0.7554 - accuracy: 0.7500
Epoch 19: val_loss did not improve from 0.88526
8/8 [==============================] - 3s 415ms/step - loss: 0.7554 - accuracy: 0.7500 - val_loss: 1.1554 - val_accuracy: 0.6500
Epoch 20/200
8/8 [==============================] - ETA: 0

In [181]:
experiment.log_model("Facial_r", filepath)
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/finetunning/b5dd8234cbfe4395ba51edc64db61beb
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [200]                : (0.550000011920929, 0.925000011920929)
COMET INFO:     batch_accuracy [200]          : (0.25, 1.0)
COMET INFO:     batch_loss [200]              : (0.11487752199172974, 2.1260457038879395)
COMET INFO:     epoch_duration [200]          : (2.7575622290005413, 8.485132381998483)
COMET INFO:     loss [200]                    : (0.295085072517395, 1.9956659078598022)
COMET INFO:     val_accuracy [200]            : (0.550000011920929, 0.8999999761581421)
COMET INFO:     val_loss [200]    